In [1]:
import datetime
from ib_insync import *
import pandas as pd

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import VRP
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from strategy_manager import StrategyManager
    from broker.functions import get_term_structure
except:
    from strategy_manager.strategies.short_vix import VRP
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from strategy_manager import StrategyManager
    from broker.functions import get_term_structure

import logging

logging.getLogger().addHandler(logging.NullHandler())



In [2]:
ib = connect_to_IB()

# Short VIX Strategy Test

In [4]:
vrp = VRP(ib,StrategyManager,TradeManager)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


/Users/jo/Desktop/lowquant/IB-Multi-Strategy-ATS/strategy_manager/strategies/short_vix.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_df['Realised Volatility'] = spx_df['Return'].rolling(21).std()*np.sqrt(252)*100


In [5]:
vrp.get_invested_contract()

Future(conId=656210801, symbol='VXM', lastTradeDateOrContractMonth='20240320', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMH4', tradingClass='VXM')

In [14]:
[pos.contract for pos in ib.portfolio() if pos.contract.symbol==vrp.instrument_symbol]

[]

In [15]:
vrp.instrument_symbol

'VXM'

In [4]:
symbol_to_short = vrp.choose_future_to_short()
contract_to_short = vrp.ib.qualifyContracts(Future(localSymbol=symbol_to_short))
current_contract = vrp.get_invested_contract()

if vrp.invested:
    print("Invested")

    #check if we need to rebalance, because we are out of investment limits
    if vrp.min_weight > abs(vrp.current_weight) or abs(vrp.current_weight) > vrp.max_weight:
        allocated_amount = vrp.equity * vrp.target_weight
        contract_price = vrp.get_contract_price(current_contract)
        print(contract_price)
        multiplier = int(current_contract.multiplier)
        target_quantity = vrp.calculate_number_of_contracts(allocated_amount,contract_price,multiplier)
        invested_quantity = [pos.position for pos in vrp.ib.portfolio() if pos.contract.localSymbol==current_contract.localSymbol][0]

        rebal_amount = (target_quantity - abs(invested_quantity))*-1

        if rebal_amount:
            print(f"We need to change our positioning by {rebal_amount} contracts")
            #vrp.trade_manager.trade(vrp,current_contract,rebal_amount)
    else:
        print(f"Within our investment limits. Our short VIX positioning is {vrp.current_weight*100}% of our portfolio")

Invested
Within our investment limits. Our short VIX positioning is -6.097035422075338% of our portfolio


In [4]:
vrp.term_structure

,Contract,LastPrice,DTE,AnnualizedYield
0,Index,12.45,0,NaN
1,VXMF4,14.34,14,-0.974896
2,VXMG4,15.44,42,-0.845960
3,VXMH4,16.35,77,-0.725212
4,VXMJ4,16.91,105,-0.655050
5,VXMK4,17.35,140,-0.579048
6,VXMM4,17.95,167,-0.550516


In [11]:
vrp.invested_contract

Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')

In [25]:
vrp.choose_future_to_short()

'VXMG4'

In [32]:

""" Get the current future contract in the portfolio """
positions = vrp.ib.portfolio()

# Sum up the investment value for all positions of the given symbol
vrp.inv_contract = [pos.contract for pos in vrp.ib.portfolio() if pos.contract.symbol=="VXM"][0]


In [37]:
today = datetime.datetime.now()
expiration_date = datetime.datetime.strptime(vrp.inv_contract.lastTradeDateOrContractMonth, '%Y%m%d')
dte = (expiration_date - today).days

if dte <=4:
    print("verkaufen")

verkaufen


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfuture; usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfuture.nj; usfuture; usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwisc

In [36]:
dte

26

In [33]:
vrp.inv_contract

Future(conId=644107700, symbol='VXM', lastTradeDateOrContractMonth='20240117', right='0', multiplier='100', primaryExchange='CFE', currency='USD', localSymbol='VXMF4', tradingClass='VXM')

In [29]:
vrp.vrp_df

,Close,Return,Realised Volatility,VIX,VRP
Date,,,,,
2023-08-23,4436.009766,NaN,NaN,15.980000,NaN
2023-08-24,4376.310059,-0.013458,NaN,17.200001,NaN
2023-08-25,4405.709961,0.006718,NaN,15.680000,NaN
2023-08-28,4433.310059,0.006265,NaN,15.080000,NaN
2023-08-29,4497.629883,0.014508,NaN,14.450000,NaN
...,...,...,...,...,...
2023-12-14,4719.549805,0.002647,6.892980,12.480000,7.267020
2023-12-15,4719.189941,-0.000076,6.939476,12.280000,7.240524
2023-12-18,4740.560059,0.004528,6.971769,12.560000,7.348231


# Place_Order func

In [9]:
contract = contract[0]
contract

Future(conId=644107700, symbol='VXM', lastTradeDateOrContractMonth='20240117', right='0', multiplier='100', primaryExchange='CFE', currency='USD', localSymbol='VXMF4', tradingClass='VXM')

In [41]:
def place_order(ib, contract,quantity, order_type='MKT',urgency='Patient',limit=None):
    '''Place an Order on the exchange via ib_insync
    :param ib: ib connection
    :param contract: ib.Contract
    :param quantity: order size as a signed integer (quantity > 0 means 'BUY'
                         and quantity < 0 means 'SELL')
    :param order_type: order type such as 'LMT', 'MKT' etc.
    :param urgency: 'Patient' (default), 'Normal', 'Urgent' 
    :param limit: if order_type 'LMT' state limit as float
    '''
    
    ib.qualifyContracts(contract)

    # Create order object
    action = quantity > 0 and 'BUY' or 'SELL'
    print(action)

    if order_type == 'LMT':
        order = LimitOrder()
        assert limit
        order.lmtPrice = float(limit)
        print(order)
    elif order_type == 'MKT':
        order = MarketOrder()
        
    order.algoStrategy = 'Adaptive'
    if urgency == 'Normal':
        order.algoParams = [TagValue('adaptivePriority', 'Normal')]
    elif urgency == 'Urgent':
        order.algoParams = [TagValue('adaptivePriority', 'Urgent')]
    else:
        order.algoParams = [TagValue('adaptivePriority', 'Patient')]
    
    order.totalQuantity = int(abs(quantity))
    
    print(type(order))
    return order
    #ib.placeOrder(contract,order)
    ib.sleep(1)

    


In [42]:
aapl = Stock('AAPL', exchange='SMART', currency='USD')
contract = ib.qualifyContracts(aapl)

In [43]:
place_order(ib,aapl,1)

TypeError: MarketOrder.__init__() missing 2 required positional arguments: 'action' and 'totalQuantity'

In [26]:
contract = Stock("AMD", 'SMART', 'USD')
order = MarketOrder("BUY", 1, algoStrategy='Adaptive', algoParams=[TagValue('adaptivePriority', 'Patient')], tif="DAY") ##algos only DAY

In [27]:
type(order)

ib_insync.order.MarketOrder

In [ ]:
ib.placeOrder()

In [51]:
ib.qualifyContracts(Future(localSymbol="VXMF4"))[0].ib_code

AttributeError: 'Future' object has no attribute 'ib_code'

[Future(conId=644107700, symbol='VXM', lastTradeDateOrContractMonth='20240117', right='0', multiplier='100', primaryExchange='CFE', currency='USD', localSymbol='VXMF4', tradingClass='VXM')]